In [1]:
import re

import pandas as pd
import pymorphy2


text_df = pd.read_csv("./data/content_description.csv", sep='\t')
text_df.head()

# разбиваем тест на слова
corpus = []
# регулярка для поиска слов
regular_expr = r'\w+'
reg_expr_compiled = re.compile(regular_expr)
# формируем датасет из отдельных слов
for raw_text in text_df.description.values:
    # приводим к нижнему регистру
    raw_text_lower = raw_text.lower()
    # разбиваем текст на слова
    text_by_words = reg_expr_compiled.findall(raw_text_lower) 
    corpus.append(text_by_words)

# нормализация текста
normalized_corpus = []
morph = pymorphy2.MorphAnalyzer()
# нормализуем каждое слово в тексте
for token_list in corpus:
    normalized_token_list = []
    for word in token_list:
        parsed_token = morph.parse(word)
        normal_form = parsed_token[0].normal_form
        normalized_token_list.append(normal_form)
    normalized_corpus.append(normalized_token_list)

# превращаем в DataFrame
doc_count = len(normalized_corpus)
doc_ids = []
tokens = []
# формируем два списка-колонки датафрейма
for doc_id in range(doc_count):
    for token in normalized_corpus[doc_id]:
        doc_ids.append(doc_id)
        tokens.append(token)

tokens_df = pd.DataFrame({
    'doc_id': doc_ids,
    'word': tokens
})
# дамми-столбец
tokens_df = tokens_df.assign(dummy = 1)
# аггрегируем статистики
word_count_df = tokens_df.groupby(['doc_id','word'])['dummy'].count().reset_index()

word_count_df[word_count_df.doc_id==0].sort_values(by='dummy', ascending=False).head(10)

,doc_id,word,dummy
6,0,в,10
28,0,и,7
36,0,который,5
110,0,шерлок,4
83,0,сериал,4
76,0,с,4
8,0,весь,3
74,0,риколетти,3
104,0,холмс,2
22,0,же,2


In [2]:
n = text_df['content'].count()
doc_frequency = word_count_df.groupby('word').count()
doc_frequency['d'] = round(doc_frequency['doc_id'] / n * 100)

doc_frequency.sort_values(by='d', ascending=False).head(10)

,doc_id,dummy,d
word,,,
и,8,8,100.0
в,8,8,100.0
на,7,7,88.0
с,7,7,88.0
он,6,6,75.0
не,6,6,75.0
свой,6,6,75.0
онлайн,6,6,75.0
что,6,6,75.0


Извлекаем Имена и Фамилии из текста

In [4]:
raw_text = text_df.description.values[3]
reg_expr = r'\w\s([A-Я]\w+.[A-Я]\w+)'
reg_expr_compiled = re.compile(reg_expr)
for g in reg_expr_compiled.findall(raw_text):
    print(g)


Стивена Фрая
Джоном Дженксом
Теда Уоллеса


Определяем жанр фильма по ключевым словам

In [ ]:
import itertools
import pandas as pd

genre_dict = {
    'комедия': ['сатирический', 'авантюрный', 'забавный'],
    'мелодрама': ['выбор', 'позор'],
    'сказка': ['приключения', 'милый', 'семейный'],
    'детектив': ['тайна', 'разгадать', 'загадочный'], 
    'триллер': ['ужас', 'зловещий', 'нерв']
}

nested_genres =[[(i, j) for j in genre_dict[i]] for i in genre_dict]
flatten_genres = list(itertools.chain(*nested_genres))
genres_df = pd.DataFrame(flatten_genres, columns = ['genre', 'character_word'])
genres_df.rename(columns = {'character_word' : 'word'}, inplace = True)
word_count_df = word_count_df.merge(genres_df)
word_count_df.groupby(['doc_id', 'genre']).count()